In [ ]:
# CNN does what it’s good at → images

# MLP does what it’s good at → tabular data

# Model learns complementary information

In [1]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping



In [2]:
df = pd.read_csv(r'Data\Dataset.csv')
df

,id,log-sigma-1-0-mm-3D_glrlm_GrayLevelNonUniformity,log-sigma-1-0-mm-3D_ngtdm_Coarseness,minor_axis_length,area,log-sigma-1-0-mm-3D_gldm_DependenceNonUniformity,log-sigma-2-0-mm-3D_ngtdm_Coarseness,log-sigma-2-0-mm-3D_gldm_DependenceNonUniformity,log-sigma-2-0-mm-3D_glrlm_GrayLevelNonUniformity,wavelet-H_ngtdm_Coarseness,...,wavelet-H_glcm_JointEntropy,wavelet-H_glrlm_GrayLevelNonUniformityNormalized,log-sigma-2-0-mm-3D_glrlm_GrayLevelNonUniformityNormalized,log-sigma-1-0-mm-3D_glrlm_HighGrayLevelRunEmphasis,log-sigma-2-0-mm-3D_glcm_MCC,LBP_hist_bin_7,log-sigma-1-0-mm-3D_gldm_HighGrayLevelEmphasis,label,unexplained_infertility,previous_miscarriages
0,1,4809.535706,0.000247,198.625804,34583,12731.61441,0.000467,13018.03444,2687.906126,0.000138,...,3.863600,0.275603,0.205812,110.158185,0.839618,0.054680,111.058150,0,0,0
1,2,4460.623751,0.000284,193.191790,32053,11900.13194,0.000526,11521.54435,2826.917297,0.000162,...,4.119202,0.252179,0.187636,72.062320,0.853366,0.063489,72.682151,0,0,0
2,3,4925.207181,0.000240,199.327172,34346,13081.91446,0.000431,13580.04041,2784.541071,0.000135,...,3.434695,0.314459,0.231505,56.084546,0.845357,0.060094,56.346329,0,0,0
3,4,4664.352654,0.000257,195.713530,34131,12645.47833,0.000490,12945.91348,2610.770111,0.000141,...,3.599379,0.297244,0.199265,72.034173,0.868814,0.058949,72.831414,0,0,0
4,5,4420.923359,0.000271,196.147021,32842,12027.18696,0.000530,11766.90250,2474.527556,0.000152,...,3.922065,0.272541,0.179913,72.190030,0.855325,0.061050,72.673284,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,124,4568.094233,0.000288,219.654077,39686,14367.67661,0.000472,14273.76697,3055.538006,0.000139,...,3.333863,0.321830,0.175555,90.434190,0.869936,0.069445,90.857481,1,0,1
124,125,4664.626825,0.000275,218.066669,39662,14590.95351,0.000430,14823.92587,3175.472188,0.000136,...,3.076035,0.349690,0.193674,72.112061,0.883472,0.069538,72.611845,1,1,0
125,126,5203.038054,0.000240,229.486021,41192,14904.76695,0.000434,14441.28705,3412.583833,0.000128,...,4.018386,0.262754,0.175031,72.478047,0.846720,0.072174,72.935619,1,0,1
126,127,5362.058473,0.000235,227.372063,40050,14744.51181,0.000405,14445.17743,3592.559575,0.000131,...,4.098497,0.255446,0.195726,110.253146,0.847308,0.059026,110.723695,1,1,0


In [3]:
tabular_features = df.drop(['id', 'label'], axis=1)
labels = df['label']


In [4]:
Scaler = StandardScaler()
x_tabular = Scaler.fit_transform(tabular_features)
y = labels.values

In [5]:
height, width = 256, 256 
batch_size=8

In [6]:
image_folder = r"C:\Users\wesal\OneDrive\Desktop\Junior-fall\Machine Learning\Uterus-Abnormality-Detection\Ultrasound-Based-Classification-of-Uterine-Abnormalities\data\images"


image_list = []

for img_id in df['id']:
    img_name = f"{int(img_id):04d}" 
    img_name += '.jpg'
    
    img_path = os.path.join(image_folder, img_name)
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    
    if img is None:
        print(f"Warning: image not found -> {img_path}")
        continue
    
    img = img / 255.0
    
    img = img[..., np.newaxis]
    
    image_list.append(img)

X_img = np.array(image_list, dtype=np.float32)
print("Images loaded:", X_img.shape)

Images loaded: (128, 256, 256, 1)


In [7]:
y = labels.values if hasattr(labels, "values") else labels

X_img_train, X_img_val, X_tab_train, X_tab_val, y_train, y_val = train_test_split(
    X_img, x_tabular, y, test_size=0.2, random_state=42, stratify=y
)
print("Training samples:", X_img_train.shape[0])
print("Validation samples:", X_img_val.shape[0])


Training samples: 102
Validation samples: 26


In [8]:
image_input = Input(shape=(256, 256, 1))
x = Conv2D(32, (3,3), activation='relu', padding='same')(image_input)
x = MaxPooling2D((2,2))(x)
x = Conv2D(64, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2))(x)
x = Conv2D(128, (3,3), activation='relu', padding='same')(x)
x = MaxPooling2D((2,2))(x)
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)

# MLP for tabular
tab_input = Input(shape=(x_tabular.shape[1],))
y = Dense(128, activation='relu')(tab_input)
y = Dropout(0.3)(y)
y = Dense(64, activation='relu')(y)

#fusion
combined = concatenate([x, y])
z = Dense(64, activation='relu')(combined)
z = Dropout(0.3)(z)
output = Dense(1, activation='sigmoid')(z)

model = Model(inputs=[image_input, tab_input], outputs=output)
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 256,  │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 256, 256,  │        320 │ input_layer[0][0] │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 128, 128,  │          0 │ conv2d[0][0]      │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │     18,496 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 64, 64,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 64, 64,    │     73,856 │ max_pooling2d_1[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 32, 32,    │          0 │ conv2d_2[0][0]    │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_layer_1       │ (None, 400)       │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ flatten (Flatten)   │ (None, 131072)    │          0 │ max_pooling2d_2[… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     51,328 │ input_layer_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │ 16,777,344 │ flatten[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 128)       │          0 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      8,256 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 192)       │          0 │ dropout[0][0],    │
│ (Concatenate)       │                   │            │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 64)        │     12,352 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 64)        │          0 │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 1)         │         65 │ dropout_2[0][0]   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 16,942,017 (64.63 MB)

 Trainable params: 16,942,017 (64.63 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
checkpoint = ModelCheckpoint(
    'best_model.h5',
    monitor='val_loss', 
    save_best_only=True,  
    verbose=1
)
early_stop = EarlyStopping(
    monitor='val_loss',
    patience=10,       
    restore_best_weights=True,
    verbose=1
)


In [10]:
history = model.fit(
    [X_img_train, X_tab_train],
    y_train,
    validation_data=([X_img_val, X_tab_val], y_val),
    epochs=50,
    batch_size=4,
    callbacks=[checkpoint, early_stop]
)


Epoch 1/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 645ms/step - accuracy: 0.5246 - loss: 0.6860
Epoch 1: val_loss improved from None to 0.62610, saving model to best_model.h5


26/26 ━━━━━━━━━━━━━━━━━━━━ 29s 936ms/step - accuracy: 0.5196 - loss: 0.6795 - val_accuracy: 0.7308 - val_loss: 0.6261
Epoch 2/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 474ms/step - accuracy: 0.5630 - loss: 0.6913
Epoch 2: val_loss improved from 0.62610 to 0.58238, saving model to best_model.h5


26/26 ━━━━━━━━━━━━━━━━━━━━ 33s 597ms/step - accuracy: 0.5980 - loss: 0.6924 - val_accuracy: 0.6923 - val_loss: 0.5824
Epoch 3/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 466ms/step - accuracy: 0.5386 - loss: 0.6789
Epoch 3: val_loss did not improve from 0.58238
26/26 ━━━━━━━━━━━━━━━━━━━━ 13s 507ms/step - accuracy: 0.5980 - loss: 0.6366 - val_accuracy: 0.7692 - val_loss: 0.5923
Epoch 4/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 480ms/step - accuracy: 0.6157 - loss: 0.5994
Epoch 4: val_loss improved from 0.58238 to 0.55949, saving model to best_model.h5


26/26 ━━━━━━━━━━━━━━━━━━━━ 16s 606ms/step - accuracy: 0.6667 - loss: 0.6174 - val_accuracy: 0.6923 - val_loss: 0.5595
Epoch 5/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 482ms/step - accuracy: 0.7641 - loss: 0.5187
Epoch 5: val_loss improved from 0.55949 to 0.51981, saving model to best_model.h5


26/26 ━━━━━━━━━━━━━━━━━━━━ 20s 582ms/step - accuracy: 0.7157 - loss: 0.5508 - val_accuracy: 0.7692 - val_loss: 0.5198
Epoch 6/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 470ms/step - accuracy: 0.6811 - loss: 0.6620
Epoch 6: val_loss did not improve from 0.51981
26/26 ━━━━━━━━━━━━━━━━━━━━ 13s 515ms/step - accuracy: 0.7255 - loss: 0.5968 - val_accuracy: 0.7692 - val_loss: 0.5392
Epoch 7/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 0.7300 - loss: 0.5500
Epoch 7: val_loss improved from 0.51981 to 0.49727, saving model to best_model.h5


...

26/26 ━━━━━━━━━━━━━━━━━━━━ 15s 576ms/step - accuracy: 0.8627 - loss: 0.2959 - val_accuracy: 0.7692 - val_loss: 0.3725
Epoch 14/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step - accuracy: 0.9110 - loss: 0.2663
Epoch 14: val_loss did not improve from 0.37255
26/26 ━━━━━━━━━━━━━━━━━━━━ 19s 529ms/step - accuracy: 0.8725 - loss: 0.2872 - val_accuracy: 0.7308 - val_loss: 0.3814
Epoch 15/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step - accuracy: 0.9232 - loss: 0.2145
Epoch 15: val_loss improved from 0.37255 to 0.34477, saving model to best_model.h5


26/26 ━━━━━━━━━━━━━━━━━━━━ 22s 581ms/step - accuracy: 0.9020 - loss: 0.2291 - val_accuracy: 0.7692 - val_loss: 0.3448
Epoch 16/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 474ms/step - accuracy: 0.9380 - loss: 0.2090
Epoch 16: val_loss improved from 0.34477 to 0.31045, saving model to best_model.h5


26/26 ━━━━━━━━━━━━━━━━━━━━ 15s 574ms/step - accuracy: 0.9412 - loss: 0.1967 - val_accuracy: 0.8077 - val_loss: 0.3104
Epoch 17/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 487ms/step - accuracy: 0.9260 - loss: 0.2583
Epoch 17: val_loss did not improve from 0.31045
26/26 ━━━━━━━━━━━━━━━━━━━━ 14s 547ms/step - accuracy: 0.9412 - loss: 0.1979 - val_accuracy: 0.8462 - val_loss: 0.3457
Epoch 18/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 462ms/step - accuracy: 0.9360 - loss: 0.1819
Epoch 18: val_loss improved from 0.31045 to 0.30857, saving model to best_model.h5


26/26 ━━━━━━━━━━━━━━━━━━━━ 15s 584ms/step - accuracy: 0.9216 - loss: 0.1880 - val_accuracy: 0.8077 - val_loss: 0.3086
Epoch 19/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 465ms/step - accuracy: 0.9900 - loss: 0.0933
Epoch 19: val_loss did not improve from 0.30857
26/26 ━━━━━━━━━━━━━━━━━━━━ 13s 506ms/step - accuracy: 0.9510 - loss: 0.1383 - val_accuracy: 0.7692 - val_loss: 0.4155
Epoch 20/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 485ms/step - accuracy: 0.9458 - loss: 0.1672
Epoch 20: val_loss did not improve from 0.30857
26/26 ━━━━━━━━━━━━━━━━━━━━ 14s 546ms/step - accuracy: 0.9510 - loss: 0.1702 - val_accuracy: 0.8077 - val_loss: 0.4348
Epoch 21/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 468ms/step - accuracy: 0.9491 - loss: 0.1435
Epoch 21: val_loss improved from 0.30857 to 0.29803, saving model to best_model.h5


26/26 ━━━━━━━━━━━━━━━━━━━━ 16s 622ms/step - accuracy: 0.9510 - loss: 0.1435 - val_accuracy: 0.8846 - val_loss: 0.2980
Epoch 22/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 393ms/step - accuracy: 0.9737 - loss: 0.1254
Epoch 22: val_loss did not improve from 0.29803
26/26 ━━━━━━━━━━━━━━━━━━━━ 11s 415ms/step - accuracy: 0.9706 - loss: 0.1110 - val_accuracy: 0.8077 - val_loss: 0.3206
Epoch 23/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 457ms/step - accuracy: 0.9171 - loss: 0.1466
Epoch 23: val_loss did not improve from 0.29803
26/26 ━━━━━━━━━━━━━━━━━━━━ 13s 505ms/step - accuracy: 0.9608 - loss: 0.1081 - val_accuracy: 0.7692 - val_loss: 0.4789
Epoch 24/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 484ms/step - accuracy: 0.9186 - loss: 0.2334
Epoch 24: val_loss did not improve from 0.29803
26/26 ━━━━━━━━━━━━━━━━━━━━ 14s 546ms/step - accuracy: 0.9608 - loss: 0.1523 - val_accuracy: 0.8077 - val_loss: 0.3507
Epoch 25/50
26/26 ━━━━━━━━━━━━━━━━━━━━ 0s 472ms/step - accuracy: 0.9632 - loss: 0.1165
Epoch 25: val_loss did not improve f

In [11]:
val_loss, val_acc = model.evaluate([X_img_val, X_tab_val], y_val)
print("Validation accuracy:", val_acc)
y_pred = model.predict([X_img_val, X_tab_val])
y_pred = (y_pred > 0.5).astype(int)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 646ms/step - accuracy: 0.8846 - loss: 0.2980
Validation accuracy: 0.8846153616905212
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 787ms/step


In [53]:
from tensorflow.keras.models import load_model

best_model = load_model('best_model.h5')
